In [ ]:
# This code was rather difficult to operate, because I had to modify my clunky Excel spreadsheet every time
# I generated a new dataset from either Ben's code or the MOSAIC program.  I decided to write a new set of
# code in Ben_MOSAIC_Comparison in order to modify direct outputs from either code into an easily manageable
# dataset similar to the random trajectory Numpy arrays I already generate.

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D
from operator import itemgetter
from random_traj import randaniso, randconv, multrandaniso, multrandconv, randsamp, multrandsamp, randall, multrandall
from random_traj import randanisorot, multrandanisorot
from ellipsoid import maxtraj, mvee, plot_mvee, plot_mveeoverl, rotmat, rotmat2, plot_mveeoverl2
from ellipsoid import ellipsax, MSDS, andiff, plot_anisoMSDS, plot_anisodiff
import random
import numpy as np
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D
from ellipsoid import extrema, enclosed_MSD

pi = np.pi
sin = np.sin
cos = np.cos

In [3]:
trajectory = np.genfromtxt('../sample_data/Prelim_MOSAIC_2DData.csv',
            delimiter =",")
trajectory=np.delete(trajectory, np.s_[0:10],0)

trajectory1 = np.genfromtxt('../sample_data/Prelim_BenProgram_2DDatab.csv',
            delimiter =",")
trajectory1=np.delete(trajectory1, 0,0)

In [ ]:
plot_3Doverlay(trajectory, 1, 24, 1, 'practice', 50, 50, 50)
plt.show()

In [ ]:
help(plot_3Doverlay)

In [10]:
trajectory1[0:10,32]

array([  0.        ,  25.3279067 ,  25.3279067 ,  25.3279067 ,
        51.42772748,  24.95821786,  10.45675768,   4.2060074 ,
         5.83955297,   6.12970809])

In [28]:

"""
A modified version of plot_3Doverlay to do 2D plots for 2D datasets. May be a little clunky.
Uses all the same variables.
"""

# Creates an array 'particles' that contains the particle number at each frame.
particles = trajectory[:, 1]
position = trajectory[:, 24:24+4]
total = int(max(particles))
total1 = total + 1
path = dict()

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    path[num] = (position[min1:max1, :])

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# Plots individual trajectories
for num in range(1, total1):

    ax.plot(path[num][:, 0], path[num][:, 1], label='Particle {}'.format(num))

axbox = ax.get_position()

legunit = max(trajectory[:,1])
ax.legend(loc=(0.85, 0.96-0.0273*legunit), prop={'size': 20})
ax.locator_params(nbins=6)

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(20)

ax.title.set_fontsize(35)
ax.tick_params(direction='out', pad=3)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().set_xlim([-40, 40])
plt.gca().set_ylim([-40, 40])

# Save your figure
plt.savefig('{}.png'.format('Prelim_MosaicTraj'), bbox_inches='tight')

plt.show()

In [29]:
"""
A modified version of plot_3Doverlay to do 2D plots for 2D datasets. May be a little clunky.
Uses all the same variables.
"""

# Creates an array 'particles' that contains the particle number at each frame.
particles = trajectory1[:, 1]
position = trajectory1[:, 23:23+4]
total = int(max(particles))
total1 = total + 1
path = dict()

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    path[num] = (position[min1:max1, :])

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# Plots individual trajectories
for num in range(1, total1):

    ax.plot(path[num][:, 0], path[num][:, 1], label='Particle {}'.format(num))

axbox = ax.get_position()

legunit = max(trajectory1[:,1])
ax.legend(loc=(0.85, 0.96-0.0273*legunit), prop={'size': 20})
ax.locator_params(nbins=6)

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(20)

ax.title.set_fontsize(35)
ax.tick_params(direction='out', pad=3)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().set_xlim([-40, 40])
plt.gca().set_ylim([-40, 40])

# Save your figure
plt.savefig('{}.png'.format('Prelim_BenTraj'), bbox_inches='tight')

plt.show()

In [30]:
"""
A modified version of plot_MSDorDeff in order to compare the 2D xy MSDs or Deffs 
from Bens code and the MOSIAC code.
"""

# Creates an array 'particles' that contains the particle number at each frame.
particles = trajectory[:, 1]
total = int(max(particles))
total1 = total + 1
rawtime = trajectory[:, 4]
rawMSD = trajectory[:, 34]
MSD = dict()
time = dict()

particles1 = trajectory1[:, 1]
totala = int(max(particles1))
total1a = totala + 1
rawtime1 = trajectory1[:, 4]
rawMSD1 = trajectory1[:, 33]
MSD1 = dict()
time1 = dict()

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    MSD[num] = (rawMSD[min1+1:min1+50])
    time[num] = (rawtime[min1+1:min1+50])

MMSD = MSD[1]
for num in range(2, total1):
    MMSD = MMSD + MSD[num]

MMSD = MMSD/total1

for num in range(1, total1a):

    hold1 = np.where(particles1 == num)
    itindex1 = hold1[0]
    min1a = min(itindex1)
    max1a = max(itindex1)
    MSD1[num] = (rawMSD1[min1a+1:min1a+60])
    time1[num] = (rawtime1[min1a+1:min1a+60])

MMSD1 = MSD1[1]
for num in range(2, total1a):
    MMSD1 = MMSD1 + MSD1[num]

MMSD1 = MMSD1/total1a

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[1][:], MMSD[:], label='MOSAIC')
ax.plot(time1[1][:], MMSD1[:], label='Ben')

ax.legend(loc=(0.85, 0.85), prop={'size': 20})

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(30)

ax.title.set_fontsize(55)
ax.set_xlabel('Time (s)')
ax.set_ylabel('MSD')
ax.tick_params(direction='out', pad=5)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(1)))

# Save your figure
plt.savefig('{}.png'.format('Prelim_CompareMSDSBenMOSAIC'), bbox_inches='tight')

plt.show()

In [20]:
help(np.delete)

Help on function delete in module numpy.lib.function_base:

delete(arr, obj, axis=None)
    Return a new array with sub-arrays along an axis deleted. For a one
    dimensional array, this returns those entries not returned by
    `arr[obj]`.
    
    Parameters
    ----------
    arr : array_like
      Input array.
    obj : slice, int or array of ints
      Indicate which sub-arrays to remove.
    axis : int, optional
      The axis along which to delete the subarray defined by `obj`.
      If `axis` is None, `obj` is applied to the flattened array.
    
    Returns
    -------
    out : ndarray
        A copy of `arr` with the elements specified by `obj` removed. Note
        that `delete` does not occur in-place. If `axis` is None, `out` is
        a flattened array.
    
    See Also
    --------
    insert : Insert elements into an array.
    append : Append elements at the end of an array.
    
    Notes
    -----
    Often it is preferable to use a boolean mask. For example:
   

In [16]:
MMSD1

array([  1.58299417,   1.58299417,   1.58299417,   3.21423297,
         1.55988862,   0.65354736,   0.26287546,   0.36497206,
         0.38310676,   1.46509774,   4.26621392,   5.49519527,
         5.49519527,   5.49519527,   5.49519527,   9.57086834,
        12.45742191,  13.38951496,  15.13655128,  15.13655128,
        19.18036384,  18.38188416,  31.98403658,  31.98403658,
        34.9274445 ,  36.56794593,  50.91581209,  50.91581209,
        47.84875071,  51.32974456,  49.14499332,  32.5510616 ,
        32.5510616 ,  27.98904373,  22.99587932,  18.73207547,
        19.71979404,  14.14521239,   9.78745215,  19.51473293,
        16.78444109,  12.40908872,  12.40908872,  12.40908872,
        10.54749248,  10.63940764,   6.08408977,   9.62162549,  15.0594293 ])